# Preliminaries

In [7]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Oct  8 22:29:59 2021

@author: ptbowen
"""
import sys
sys.path.append('..')
import worx.ElectromagneticObjects as emag
import worx.MathObjects as math
import worx.Geometries as geom
import worx.PlotTools as PlotTools
import worx.HFSS_IO as HFSS_IO

import numpy as np
import matplotlib.pyplot as plt
from numpy.lib.scimath import sqrt
from mayavi import mlab


# constants
pi=np.pi
Z0=376
c=2.998e8

# We are going to operate in a 3D euclidean global geometry.
(xhat,yhat,zhat)=math.CreateEuclideanBasis()
vec_space=[xhat,yhat,zhat]

# Creation of a single waveguide with a dipole array

Setup geometry and extract polarizability:

In [12]:
# Waveguide design
f0=9.8e9
lmbda=c/f0
n_wg=sqrt(3.55*(1-0.0027j))
pitch=0.008
N=5
L=N*pitch
a=14e-3
b=0.762e-3
WG=geom.Waveguide(a,b,n_wg,L)
WG.N_modes=1

# Create a half space for far-field calcs
HS=geom.HalfSpace()


# Data files for polarizability extraction
S11MagFile='../HFSS/S11mag_MikeCell_DS_1p1.csv'
S11PhaseFile='../HFSS/S11phase_MikeCell_DS_1p1.csv'
S21MagFile='../HFSS/S21mag_MikeCell_DS_1p1.csv'
S21PhaseFile='../HFSS/S21phase_MikeCell_DS_1p1.csv'
HFSS_Files={'S11Mag':S11MagFile,'S11Phase':S11PhaseFile,
            'S21Mag':S21MagFile,'S21Phase':S21PhaseFile}


# Individual dipole setup for sim
r0=xhat*1.6e-3+yhat*0+zhat*L/2
dip=emag.Dipole(r0)
dip.extract(WG,HFSS_Files)


# Dipole array design
z_pos=np.arange(pitch/2,(N+1/2)*pitch,pitch)+1.2e-3
x_pos=np.ones(np.shape(z_pos))*dip.r0[xhat]
y_pos=np.ones(np.shape(z_pos))*dip.r0[yhat]
r_pos=math.GridToListOfVectors({xhat:x_pos,yhat:y_pos,zhat:z_pos})
dip_array=dip.Array(r_pos)
for dipole in dip_array:
    WG.add_dipole(dipole)
    HS.add_dipole(dipole)
    
# Now, position it where you want it in space
R=math.RotTensor(pi/2,'x',vec_space)
WG.Rotate(R)
WG.Translate(xhat*-a/2)

# Plot geometry
mlab.init_notebook()
mlab.clf(1)
WG.MayaviPlot(1,plot_fields=False,field_clip=30)
PlotTools.MayaviAxes(10,-L/3,L/3,0,L,-L/10,L/2)
mlab.figure(figure=1,bgcolor=(1,1,1), fgcolor=(0.,0.,0.))

Notebook initialized with ipy backend.


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01^\x08\x02\x00\x00\x00$?\xde_\x00\…

The last thing to do in the setup is to tune the dipoles to the desired frequency so that their polarizability tensors will be assigned.  Then we give a the waveguide a source and solve.  We will just choose to tune all the dipoles to the same frequency.

In this example, we will set the solver to "coupled", which means that the dipoles are coupled.  The "coupled" solver will include interactions through every geometry to which the dipoles are assigned.  So since these dipoles have been assigned to both the waveguide and the half-space, the solver will include both the interactions through the waveguide and through the half-space (i.e. over the top of the the waveguide through the half-space interaction).

In [11]:
for dipole in WG.DipoleList: dipole.tune(f0)
WG.Source(f0)
WG.Compute(f0,compute_type='coupled')
HS.ComputeFarField(f0)

# Plot geometry
mlab.figure(figure=3,bgcolor=(1,1,1), fgcolor=(0.,0.,0.))
mlab.clf()
WG.MayaviPlot(1,plot_fields=True,field_clip=30)
PlotTools.MayaviAxes(10,-L/3,L/3,0,L,-L/10,L/2)

# Far field plotting
directivity=HS.FarField.directivity
k=HS.FarField.k
(theta_grid,phi_grid,th_axis,ph_axis,dth,dph)=math.HalfSphereGrid(360,90)
Dir_dB=10*np.log10(np.real(directivity(k(theta_grid,phi_grid)))+1e-16) 
Dir_dB[Dir_dB<0]=0
rhat=math.SphericalBasis('r')
X=Dir_dB*(rhat.dot(xhat))(theta_grid,phi_grid)
Y=Dir_dB*(rhat.dot(yhat))(theta_grid,phi_grid)
Z=Dir_dB*(rhat.dot(zhat))(theta_grid,phi_grid)

mlab.init_notebook()
mlab.clf(2)
mlab.mesh(X,Y,Z)
coordmax=np.max([X,Y,Z])
PlotTools.MayaviAxes(10,-coordmax,coordmax,-coordmax,coordmax,0,coordmax)
mlab.figure(figure=2,bgcolor=(1,1,1), fgcolor=(0.,0.,0.))

Notebook initialized with ipy backend.


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01\x90\x00\x00\x01^\x08\x02\x00\x00\x00$?\xde_\x00\…